<a href="https://colab.research.google.com/github/Noob919/IoT_Security_Project/blob/main/MultiClassification_for_non_DDoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [3]:
import os
import dask.dataframe as dd

# Set the dataset directory
DATASET_DIRECTORY = '/content/drive/MyDrive/Data/'

# Importing Dataset

df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets) * 0.8)]
test_sets = df_sets[int(len(df_sets) * 0.8):]

# Create a Dask DataFrame for training data
training_dataframes = [dd.read_csv(DATASET_DIRECTORY + train_set) for train_set in training_sets]

# Concatenate all Dask DataFrames into a single training Dask DataFrame
training_data = dd.concat(training_dataframes, axis=0)

In [ ]:
# List of label values you want to filter on
label_values_to_select = [
    'DDoS-RSTFINFlood',
    'DoS-TCP_Flood',
    'DDoS-ICMP_Flood',
    'DoS-UDP_Flood',
    'DoS-SYN_Flood',
    'Mirai-greeth_flood',
    'DDoS-SynonymousIP_Flood',
    'Mirai-udpplain',
    'DDoS-SYN_Flood',
    'DDoS-PSHACK_Flood',
    'DDoS-TCP_Flood',
    'DDoS-UDP_Flood',
    'BenignTraffic',
    'MITM-ArpSpoofing',
    'DDoS-ACK_Fragmentation',
    'Mirai-greip_flood',
    'DoS-HTTP_Flood',
    'DDoS-ICMP_Fragmentation',
    'Recon-PortScan',
    'DNS_Spoofing',
    'DDoS-UDP_Fragmentation',
    'Recon-OSScan',
    'XSS',
    'DDoS-HTTP_Flood',
    'Recon-HostDiscovery',
    'CommandInjection',
    'VulnerabilityScan',
    'DDoS-SlowLoris',
    'Backdoor_Malware',
    'BrowserHijacking',
    'DictionaryBruteForce',
    'SqlInjection',
    'Recon-PingSweep',
    'Uploading_Attack'
]

# Create an empty list to store selected rows
selected_rows = []

# Loop through each label value and select the first 300 rows for each
for label_value in label_values_to_select:
    label_rows = training_data[training_data['label'] == label_value].head(800)
    selected_rows.append(label_rows)

# Concatenate the selected rows into a single DataFrame
selected_data = dd.concat(selected_rows, axis=0)

# **Data to be test**

In [18]:
pd.read_csv("/content/pred_df.csv")

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,z_test,new_label,Binary Prediction
0,0.000000,0.00,1.00,64.00,2.982356,2.982356,0.0,0.0,0.0,0.0,...,8.312453e+07,9.5,9.165151,0.000000,0.000000e+00,0.00,141.55,DDoS-ICMP_Flood,DDos,DDos
1,2.034678,78802.70,6.00,199.60,44.511043,44.511043,0.0,0.0,0.0,0.0,...,2.289295e-03,5.5,79.508497,1410.958754,1.253443e+06,0.90,38.50,Recon-OSScan,other,DDos
2,0.000000,0.00,47.00,64.00,84.416214,84.416214,0.0,0.0,0.0,0.0,...,8.357402e+07,9.5,34.000000,0.000000,0.000000e+00,0.00,141.55,Mirai-greip_flood,other,other
3,0.000000,0.00,46.53,63.36,3.447036,3.447036,0.0,0.0,0.0,0.0,...,8.357373e+07,9.5,33.975197,4.949597,4.083680e+02,0.03,141.55,Mirai-greip_flood,other,other
4,0.000000,54.00,6.00,64.00,2.437226,2.437226,0.0,0.0,0.0,0.0,...,8.306742e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,DDoS-TCP_Flood,DDos,DDos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,5.336254,108.00,6.00,64.00,0.374795,0.374795,0.0,0.0,0.0,0.0,...,8.295583e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,DoS-TCP_Flood,DDos,DDos
2066,0.000000,0.74,46.59,64.58,18.437595,18.437595,0.0,0.0,0.0,0.0,...,8.367716e+07,9.5,34.254725,25.165849,2.476303e+03,0.13,141.55,Mirai-greeth_flood,other,other
2067,171.330202,4096094.00,6.00,64.00,22.479983,22.479983,0.0,0.0,0.0,0.0,...,1.666040e+08,13.5,12.489996,28.924370,4.219096e+02,1.00,244.60,MITM-ArpSpoofing,other,other
2068,30.435525,210748.10,5.40,74.00,47.119139,47.119139,0.0,0.0,0.0,0.0,...,3.574085e-03,5.5,11.905508,12.401621,1.691517e+02,0.90,38.50,BenignTraffic,other,other


# Decision Tree

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Decision Tree)
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Fit the Decision Tree classifier on the scaled training data
decision_tree_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = decision_tree_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['Mirai-greip_flood' 'Mirai-greip_flood' 'MITM-ArpSpoofing' 'DNS_Spoofing'
 'BenignTraffic' 'Mirai-udpplain' 'Mirai-udpplain' 'BenignTraffic'
 'Recon-PortScan' 'DNS_Spoofing' 'Mirai-greeth_flood' 'Recon-OSScan'
 'Mirai-greeth_flood' 'Recon-OSScan' 'Recon-OSScan' 'Recon-HostDiscovery'
 'DNS_Spoofing' 'Mirai-udpplain' 'Recon-OSScan' 'Recon-OSScan'
 'DNS_Spoofing' 'Recon-OSScan' 'DDoS-SynonymousIP_Flood' 'Recon-PortScan'
 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-greip_flood' 'Recon-PortScan'
 'BenignTraffic' 'Recon-HostDiscovery' 'DNS_Spoofing' 'Recon-PortScan'
 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-udpplain' 'Recon-OSScan'
 'Recon-OSScan' 'BenignTraffic' 'Recon-PortScan' 'Recon-PortScan'
 'BenignTraffic' 'Mirai-udpplain' 'DNS_Spoofing' 'MITM-ArpSpoofing'
 'Mirai-udpplain' 'Recon-OSScan' 'Recon-HostDiscovery' 'Mirai-greip_flood'
 'Mirai-udpplain' 'Recon-HostDiscovery' 'Mirai-udpplain' 'DNS_Spoofing'
 'Recon-HostDiscovery' 'Mirai-greip_flood' 'Recon-OSScan'
 'Recon-HostDiscovery' 'MITM-ArpS

# MLP

In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Multi-Layer Perceptron)
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Fit the MLP classifier on the scaled training data
mlp_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = mlp_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['Mirai-greip_flood' 'Mirai-greip_flood' 'BenignTraffic' 'DNS_Spoofing'
 'BenignTraffic' 'Mirai-udpplain' 'Mirai-udpplain' 'BenignTraffic'
 'Recon-PortScan' 'Recon-PortScan' 'Mirai-greeth_flood' 'Recon-OSScan'
 'Mirai-greeth_flood' 'Recon-OSScan' 'BenignTraffic' 'Recon-HostDiscovery'
 'MITM-ArpSpoofing' 'Mirai-udpplain' 'BenignTraffic' 'Recon-OSScan'
 'DNS_Spoofing' 'Recon-PortScan' 'DDoS-SynonymousIP_Flood'
 'Recon-PortScan' 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-greip_flood'
 'Recon-OSScan' 'BenignTraffic' 'Recon-HostDiscovery' 'DNS_Spoofing'
 'Recon-PortScan' 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-udpplain'
 'Recon-OSScan' 'Recon-OSScan' 'BenignTraffic' 'Recon-PortScan'
 'Recon-PortScan' 'BenignTraffic' 'Mirai-udpplain' 'DNS_Spoofing'
 'MITM-ArpSpoofing' 'Mirai-udpplain' 'Recon-OSScan' 'Recon-HostDiscovery'
 'Mirai-greip_flood' 'Mirai-udpplain' 'Recon-HostDiscovery'
 'Mirai-udpplain' 'Recon-HostDiscovery' 'Recon-HostDiscovery'
 'Mirai-greip_flood' 'Recon-OSScan' 'Recon-HostDiscovery

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVM

In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Support Vector Machine)
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

# Fit the SVM classifier on the scaled training data
svm_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = svm_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['Mirai-greip_flood' 'Mirai-greip_flood' 'BenignTraffic' 'BenignTraffic'
 'BenignTraffic' 'Mirai-udpplain' 'Mirai-udpplain' 'BenignTraffic'
 'Recon-OSScan' 'Recon-PortScan' 'Mirai-greeth_flood'
 'Recon-HostDiscovery' 'Mirai-greeth_flood' 'Recon-PortScan'
 'BenignTraffic' 'Recon-HostDiscovery' 'DNS_Spoofing' 'Mirai-udpplain'
 'BenignTraffic' 'Recon-HostDiscovery' 'DNS_Spoofing' 'Recon-PortScan'
 'DoS-SYN_Flood' 'Recon-HostDiscovery' 'Mirai-udpplain' 'BenignTraffic'
 'Mirai-greip_flood' 'Recon-PortScan' 'BenignTraffic'
 'Recon-HostDiscovery' 'Recon-OSScan' 'Recon-HostDiscovery'
 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-udpplain' 'Recon-PortScan'
 'BenignTraffic' 'BenignTraffic' 'BenignTraffic' 'BenignTraffic'
 'BenignTraffic' 'Mirai-udpplain' 'Recon-HostDiscovery' 'DNS_Spoofing'
 'Mirai-udpplain' 'Recon-HostDiscovery' 'Recon-HostDiscovery'
 'Mirai-greip_flood' 'Mirai-udpplain' 'Recon-HostDiscovery'
 'Mirai-udpplain' 'Recon-HostDiscovery' 'Recon-HostDiscovery'
 'Mirai-greip_flood' 'Recon-Po

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Random Forest**

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Random Forest)
multiclass_classifier = RandomForestClassifier(n_estimators=1000, random_state=42)

# Fit the multiclass classifier on the scaled training data
multiclass_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Fit and transform the training data
final_x_scaled = numeric_transformer.fit_transform(final_X)

# Predict on the scaled test data
multiclass_predictions = multiclass_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification recall Score: {multiclass_f1_score:.2f}")

['Mirai-greip_flood' 'Mirai-greip_flood' 'DDoS-ICMP_Flood'
 'DDoS-ICMP_Flood' 'DDoS-ICMP_Flood' 'Mirai-udpplain' 'Mirai-udpplain'
 'MITM-ArpSpoofing' 'DDoS-ICMP_Flood' 'Recon-PortScan' 'Mirai-greip_flood'
 'Recon-HostDiscovery' 'Mirai-greip_flood' 'DDoS-ICMP_Flood'
 'DDoS-ICMP_Flood' 'DDoS-ICMP_Flood' 'DoS-UDP_Flood' 'Mirai-udpplain'
 'DoS-UDP_Flood' 'DDoS-ICMP_Flood' 'DoS-UDP_Flood' 'DDoS-ICMP_Flood'
 'DDoS-ICMP_Flood' 'DDoS-ICMP_Flood' 'Mirai-udpplain' 'MITM-ArpSpoofing'
 'Mirai-greip_flood' 'Recon-HostDiscovery' 'DDoS-ICMP_Fragmentation'
 'DDoS-ICMP_Flood' 'MITM-ArpSpoofing' 'DDoS-ICMP_Flood' 'Mirai-udpplain'
 'DoS-UDP_Flood' 'Mirai-udpplain' 'DoS-UDP_Flood'
 'DDoS-ICMP_Fragmentation' 'DoS-UDP_Flood' 'Mirai-greeth_flood'
 'DDoS-ICMP_Fragmentation' 'MITM-ArpSpoofing' 'Mirai-udpplain'
 'DDoS-ICMP_Flood' 'Recon-PortScan' 'Mirai-udpplain' 'DDoS-ICMP_Flood'
 'DDoS-ICMP_Flood' 'Mirai-greip_flood' 'Mirai-udpplain' 'DoS-UDP_Flood'
 'Mirai-udpplain' 'DDoS-ICMP_Flood' 'DDoS-ICMP_Flood' 'Mirai

# XGBoost

In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Encode the target labels using LabelEncoder
label_encoder = LabelEncoder()
y_multiclass_encoded = label_encoder.fit_transform(y_multiclass)

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (XGBoost)
xgb_classifier = xgb.XGBClassifier(random_state=42)

# Fit the XGBoost classifier on the scaled training data
xgb_classifier.fit(X_train_scaled, y_multiclass_encoded)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions_encoded = xgb_classifier.predict(final_x_scaled)

# Inverse transform the predictions to get original labels
multiclass_predictions = label_encoder.inverse_transform(multiclass_predictions_encoded)

print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['Mirai-greip_flood' 'Mirai-greip_flood' 'MITM-ArpSpoofing' 'DNS_Spoofing'
 'BenignTraffic' 'Mirai-udpplain' 'Mirai-udpplain' 'BenignTraffic'
 'Recon-PortScan' 'DNS_Spoofing' 'Mirai-greeth_flood' 'Recon-OSScan'
 'Mirai-greeth_flood' 'Recon-OSScan' 'Recon-OSScan' 'Recon-HostDiscovery'
 'DNS_Spoofing' 'Mirai-udpplain' 'Recon-OSScan' 'Recon-OSScan'
 'DNS_Spoofing' 'Recon-OSScan' 'DDoS-SynonymousIP_Flood' 'Recon-PortScan'
 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-greip_flood' 'Recon-PortScan'
 'BenignTraffic' 'Recon-HostDiscovery' 'DNS_Spoofing' 'Recon-PortScan'
 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-udpplain' 'Recon-OSScan'
 'Recon-OSScan' 'BenignTraffic' 'Recon-PortScan' 'Recon-PortScan'
 'BenignTraffic' 'Mirai-udpplain' 'DNS_Spoofing' 'MITM-ArpSpoofing'
 'Mirai-udpplain' 'Recon-OSScan' 'Recon-HostDiscovery' 'Mirai-greip_flood'
 'Mirai-udpplain' 'Recon-HostDiscovery' 'Mirai-udpplain' 'DNS_Spoofing'
 'Recon-HostDiscovery' 'Mirai-greip_flood' 'Recon-OSScan'
 'Recon-HostDiscovery' 'MITM-ArpS

## Logistic **Regression**

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Logistic Regression)
multiclass_classifier = LogisticRegression(max_iter=10000, random_state=42)

# Fit the multiclass classifier on the scaled training data
multiclass_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'other']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Fit and transform the training data
final_x_scaled = numeric_transformer.fit_transform(final_X)

# Predict on the scaled test data
multiclass_predictions = multiclass_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")


['Mirai-greip_flood' 'Mirai-greip_flood' 'BenignTraffic'
 'Recon-HostDiscovery' 'Recon-HostDiscovery' 'Mirai-udpplain'
 'Mirai-udpplain' 'BenignTraffic' 'Recon-PortScan' 'DoS-SYN_Flood'
 'Mirai-greip_flood' 'DDoS-PSHACK_Flood' 'Mirai-greip_flood'
 'DoS-HTTP_Flood' 'Recon-HostDiscovery' 'Recon-HostDiscovery'
 'DoS-UDP_Flood' 'Mirai-udpplain' 'Recon-PortScan' 'Recon-HostDiscovery'
 'DNS_Spoofing' 'Recon-PortScan' 'DDoS-SynonymousIP_Flood'
 'Recon-HostDiscovery' 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-greip_flood'
 'Recon-PortScan' 'BenignTraffic' 'Recon-HostDiscovery' 'Recon-OSScan'
 'Recon-HostDiscovery' 'Mirai-udpplain' 'Recon-OSScan' 'Mirai-udpplain'
 'Recon-PortScan' 'Recon-OSScan' 'Recon-OSScan' 'DoS-HTTP_Flood'
 'DDoS-ACK_Fragmentation' 'DDoS-ACK_Fragmentation' 'Mirai-udpplain'
 'Recon-OSScan' 'Recon-PortScan' 'Mirai-udpplain' 'Recon-HostDiscovery'
 'DDoS-PSHACK_Flood' 'Mirai-greip_flood' 'Mirai-udpplain'
 'Recon-HostDiscovery' 'Mirai-udpplain' 'Recon-HostDiscovery'
 'Recon-HostDisc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
